In [6]:
from dendron import *
from dendron.actions.generate_action import GenerateAction
from dendron.configs.hflm_config import HFLMConfig
from dendron.configs.lm_action_config import LMActionConfig

In [11]:
model_config = HFLMConfig(
    model="microsoft/Phi-3-mini-4k-instruct",
    device="cpu",
    parallelize=False,
    load_in_8bit=False
)

node_config = LMActionConfig(
    node_name="GenerateAction",
    input_key="in",
    output_key="out",
    max_new_tokens=100,
    temperature=0.0,
)

# Create and configure node
node = GenerateAction(model_config, node_config)
tree = BehaviorTree("generate-tree") 
tree.set_root(node)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
prompt = """{system}
### Instruction:
{query}

### Response:
"""

In [13]:
import re

def trunc(node):
    out_str = node.blackboard[node.output_key]
    m = re.search(r'^Yes\.', out_str)
    if m:
        node.blackboard[node.output_key] = m.group(0).rstrip('.') 

tree.root.add_post_tick(trunc)

In [14]:
tree.blackboard["in"] = prompt.format(system="You are busy and don't have much time to be verbose.", 
                                      query="The following is a headline from a newspaper: 'History-making SpaceX booster mostly destroyed in post-flight topple.' Is this about a spacecraft? Give a one-word yes or no.")

In [15]:
status = tree.tick_once()
if status == NodeStatus.SUCCESS:
     print(tree.blackboard["out"])

Both `max_new_tokens` (=100) and `max_length`(=329) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Yes


In [16]:
print(tree.blackboard)

Key              | Description      | Type             | Value            |
in               | Autogenerated en | <class 'str'>    | You are busy and | 
out              | Autogenerated en | <class 'str'>    | Yes              | 



In [17]:
tree.blackboard['out']

'Yes'